# LOG6302A — Analyse d’applications et Cyber-sécurité<br>Laboratoire #1

**Quentin Guidée (2206809), Nam Vu (2230468)**

Polytechnique Montréal – Hiver 2024


In [1]:
import re
from abc import ABC, abstractmethod
from pathlib import Path

from code_analysis import AST, ASTReader

In [2]:
CODE_TO_ANALYZE_DIR = Path("../code_to_analyze")

UNIT_TESTS_AST_DIR = CODE_TO_ANALYZE_DIR / "test_cve"

WP_AST_DIR = CODE_TO_ANALYZE_DIR / "wordpress_ast"
WP_SOURCES_DIR = CODE_TO_ANALYZE_DIR / "wordpress_sources"

## Détection de patterns

In [3]:
reader = ASTReader()

In [4]:
class AbstractVisitor(ABC):
    def __init__(self):
        self.ast: AST
        self.sources_dir: Path

    def visit(self, ast: AST, sources_dir: Path):
        self.ast = ast
        self.sources_dir = sources_dir
        self._visit(self.ast.get_root())

    @abstractmethod
    def _visit(self, node_id: int):
        ...

In [5]:
class ASTQueryVisitor(AbstractVisitor):
    def _visit(self, node_id: int):
        if self.ast.get_type(node_id) == "FunctionCall":
            image = self.ast.get_image(node_id)
            if image in ["mysql_query", "mysqli_query"]:
                print(
                    f"Database call '{image}' ",
                    f'in file "{(self.sources_dir / self.ast.get_filename())}", line {self.ast.get_position(node_id)[0]}',
                )

        elif (
            self.ast.get_type(node_id) == "BinOP"
            and self.ast.get_image(node_id) == "->"
        ):
            children = self.ast.get_children(node_id)
            if (
                len(children) > 1
                and self.ast.get_type(children[0]) == "Variable"
                and self.ast.get_type(children[1]) == "MethodCall"
                and self.ast.get_image(children[1]) == "execute"
            ):
                print(
                    f"Database call '{self.ast.get_image(children[0])}->{self.ast.get_image(children[1])}' "
                    f'in file "{(self.sources_dir / self.ast.get_filename())}", line {self.ast.get_position(node_id)[0]}',
                )

            elif (
                len(children) > 1
                and self.ast.get_type(children[0]) == "BinOP"
                and self.ast.get_type(children[1]) == "MethodCall"
                and self.ast.get_image(children[1]) == "exec"
            ):
                binop_children = self.ast.get_children(children[0])
                if (
                    len(binop_children) > 1
                    and self.ast.get_image(binop_children[1]) == "mysql"
                ):
                    print(
                        f"Database call '{self.ast.get_image(binop_children[0])}->{self.ast.get_image(binop_children[1])}->{self.ast.get_image(children[1])}' in "
                        f'in file "{(self.sources_dir / self.ast.get_filename())}", line {self.ast.get_position(node_id)[0]}',
                    )

        for child_id in self.ast.get_children(node_id):
            self._visit(child_id)

In [6]:
def run_on_wordpress(visitor: AbstractVisitor):
    with (WP_AST_DIR / "filelist").open() as file:
        for line in file:
            ast = reader.read_ast((WP_AST_DIR / line.strip()).as_posix())
            visitor.visit(ast, WP_SOURCES_DIR)

In [7]:
visitor = ASTQueryVisitor()
run_on_wordpress(visitor)

Database call 'this->mysql->exec' in in file "../code_to_analyze/wordpress_sources/wp-includes/SimplePie/Cache/MySQL.php", line 130
Database call 'this->mysql->exec' in in file "../code_to_analyze/wordpress_sources/wp-includes/SimplePie/Cache/MySQL.php", line 139
Database call 'query->execute' in file "../code_to_analyze/wordpress_sources/wp-includes/SimplePie/Cache/MySQL.php", line 168
Database call 'query->execute' in file "../code_to_analyze/wordpress_sources/wp-includes/SimplePie/Cache/MySQL.php", line 188
Database call 'query->execute' in file "../code_to_analyze/wordpress_sources/wp-includes/SimplePie/Cache/MySQL.php", line 200
Database call 'query->execute' in file "../code_to_analyze/wordpress_sources/wp-includes/SimplePie/Cache/MySQL.php", line 217
Database call 'query->execute' in file "../code_to_analyze/wordpress_sources/wp-includes/SimplePie/Cache/MySQL.php", line 239
Database call 'query->execute' in file "../code_to_analyze/wordpress_sources/wp-includes/SimplePie/Cache/M

## Detection de vulnerabilités connues

### CVE-2017-7189

In [8]:
class AST_2017_7189_Visitor(AbstractVisitor):
    def _visit(self, node_id: int):
        children = self.ast.get_children(node_id)

        if (
            len(children) > 1
            and self.ast.get_type(node_id) == "FunctionCall"
            and self.ast.get_image(node_id) == "fsockopen"
            and self.ast.get_type(children[1]) == "ArgumentList"
        ):
            params = self.ast.get_children(children[1])
            if len(params) > 0 and (
                self.ast.get_type(params[0]) != "StringExpression"
                or re.search(r"^\$|:\d+$", self.ast.get_image(params[0]))
            ):
                print(
                    f"Potential CVE-2017-7189 detected "
                    f'in file "{self.sources_dir / self.ast.get_filename()}", line {self.ast.get_position(node_id)[0]}'
                )

        for child_id in children:
            self._visit(child_id)

In [9]:
visitor = AST_2017_7189_Visitor()
ast = reader.read_ast((UNIT_TESTS_AST_DIR / "2017_7189.php.ast.json").as_posix())
visitor.visit(ast, CODE_TO_ANALYZE_DIR)

Potential CVE-2017-7189 detected in file "../code_to_analyze/test_cve/2017_7189.php", line 9


In [10]:
run_on_wordpress(visitor)

Potential CVE-2017-7189 detected in file "../code_to_analyze/wordpress_sources/wp-admin/includes/class-ftp-pure.php", line 49
Potential CVE-2017-7189 detected in file "../code_to_analyze/wordpress_sources/wp-admin/includes/class-ftp-pure.php", line 113
Potential CVE-2017-7189 detected in file "../code_to_analyze/wordpress_sources/wp-includes/IXR/class-IXR-client.php", line 90
Potential CVE-2017-7189 detected in file "../code_to_analyze/wordpress_sources/wp-includes/IXR/class-IXR-client.php", line 92
Potential CVE-2017-7189 detected in file "../code_to_analyze/wordpress_sources/wp-includes/SimplePie/File.php", line 162
Potential CVE-2017-7189 detected in file "../code_to_analyze/wordpress_sources/wp-includes/class-pop3.php", line 97
Potential CVE-2017-7189 detected in file "../code_to_analyze/wordpress_sources/wp-includes/class-snoopy.php", line 1145
Potential CVE-2017-7189 detected in file "../code_to_analyze/wordpress_sources/wp-includes/class-smtp.php", line 303


### CVE-2021-21707

In [11]:
class AST_2021_21707_Visitor(AbstractVisitor):
    def _visit(self, node_id: int):
        children = self.ast.get_children(node_id)

        if (
            len(children) >= 2
            and self.ast.get_type(node_id) == "FunctionCall"
            and self.ast.get_image(node_id) == "simplexml_load_file"
        ):
            print(
                f"Potential CVE-2021-21707 detected "
                f'in file "{self.sources_dir / self.ast.get_filename()}", line {self.ast.get_position(node_id)[0]}'
            )

        for child_id in children:
            self._visit(child_id)

In [12]:
visitor = AST_2021_21707_Visitor()
ast = reader.read_ast((UNIT_TESTS_AST_DIR / "2021_21707.php.ast.json").as_posix())
visitor.visit(ast, CODE_TO_ANALYZE_DIR)

Potential CVE-2021-21707 detected in file "../code_to_analyze/test_cve/2021_21707.php", line 10


In [13]:
run_on_wordpress(visitor)

### CVE-2019-9025

In [14]:
class AST_2019_9025_Visitor(AbstractVisitor):
    def _visit(self, node_id: int):
        children = self.ast.get_children(node_id)

        if (
            len(children) >= 2
            and self.ast.get_type(node_id) == "FunctionCall"
            and self.ast.get_image(node_id) == "mb_split"
        ):
            params = self.ast.get_children(children[1])
            # TODO: Check that the multibyte string is illegal
            print(
                f"Potential CVE-2019-9025 detected "
                f'in file "{self.sources_dir / self.ast.get_filename()}", line {self.ast.get_position(node_id)[0]}'
            )

        for child_id in children:
            self._visit(child_id)

In [15]:
visitor = AST_2019_9025_Visitor()
ast = reader.read_ast((UNIT_TESTS_AST_DIR / "2019_9025.php.ast.json").as_posix())
visitor.visit(ast, CODE_TO_ANALYZE_DIR)

Potential CVE-2019-9025 detected in file "../code_to_analyze/test_cve/2019_9025.php", line 8


In [16]:
run_on_wordpress(visitor)